# Exploring Data example with Pandas

Let’s take a quick look at what we can do with some simple data using Python. I took a look around Kaggle and found San Francisco City Employee salary data.

First, I imported the data and read it into a Panda’s dataframe. Then, I wanted to see the data. I looked at a few of the rows and used the Panda’s dataframe’s describe method to see how the data is distributed.

In [9]:
import pandas as pd

# To ensure no mixed types either set False, or specify the type with the dtype parameter
salaries = pd.read_csv("./datasets/salaries.csv", low_memory=False)

# Seting the DataFrame index using Id columns
salaries.set_index('Id', inplace=True)

salaries.head()

,EmployeeName,JobTitle,BasePay,OvertimePay,OtherPay,Benefits,TotalPay,TotalPayBenefits,Year,Notes,Agency,Status
Id,,,,,,,,,,,,
1,NATHANIEL FORD,GENERAL MANAGER-METROPOLITAN TRANSIT AUTHORITY,167411.18,0.0,400184.25,NaN,567595.43,567595.43,2011,NaN,San Francisco,NaN
2,GARY JIMENEZ,CAPTAIN III (POLICE DEPARTMENT),155966.02,245131.88,137811.38,NaN,538909.28,538909.28,2011,NaN,San Francisco,NaN
3,ALBERT PARDINI,CAPTAIN III (POLICE DEPARTMENT),212739.13,106088.18,16452.6,NaN,335279.91,335279.91,2011,NaN,San Francisco,NaN
4,CHRISTOPHER CHONG,WIRE ROPE CABLE MAINTENANCE MECHANIC,77916.0,56120.71,198306.9,NaN,332343.61,332343.61,2011,NaN,San Francisco,NaN
5,PATRICK GARDNER,"DEPUTY CHIEF OF DEPARTMENT,(FIRE DEPARTMENT)",134401.6,9737.0,182234.59,NaN,326373.19,326373.19,2011,NaN,San Francisco,NaN


In [10]:
salaries.describe()

,TotalPay,TotalPayBenefits,Year,Notes
count,148654.000000,148654.000000,148654.000000,0.0
mean,74768.321972,93692.554811,2012.522643,NaN
std,50517.005274,62793.533483,1.117538,NaN
min,-618.130000,-618.130000,2011.000000,NaN
25%,36168.995000,44065.650000,2012.000000,NaN
50%,71426.610000,92404.090000,2013.000000,NaN
75%,105839.135000,132876.450000,2014.000000,NaN
max,567595.430000,567595.430000,2014.000000,NaN


I noticed the dataset had numerous years, but I was most interested in the most recent data, so I decided to make a new dataframe of just that data.

In [14]:
# The new DataFrame for data in 2014
latest_salaries = salaries[salaries['Year'] == 2014]

latest_salaries.describe()

,TotalPay,TotalPayBenefits,Year,Notes
count,38123.000000,38123.000000,38123.0,0.0
mean,75463.918140,100250.918884,2014.0,NaN
std,51697.713029,66238.852887,0.0,NaN
min,-618.130000,-618.130000,2014.0,NaN
25%,33436.210000,44199.990000,2014.0,NaN
50%,72359.980000,101105.370000,2014.0,NaN
75%,107974.445000,142746.950000,2014.0,NaN
max,471952.640000,510732.680000,2014.0,NaN


I looked up the average yearly rental cost from the latest reports on Priceonomics. The median apartment for rent in San Francisco is $3,880 per month for a single-income household with no children in a one-bedroom apartment.

I wanted to know what percentage of their income the average city employee was paying for rent.

In [21]:
# Average money for appartment rent in one year
average_yearly_rent = 3880 * 12

# Average income of the city in the year 2014
average_city_pay = latest_salaries['TotalPay'].mean()

# Percentage of the rent above the income
percentage = average_yearly_rent/average_city_pay * 100

print('%f%%' % percentage)


61.698360%


How many city employees make below the average one-bedroom rent per year?

In [24]:
# Computing the number of employees who have the income below than the average one-bedroom rent per year
latest_salaries[latest_salaries['TotalPay'] < average_yearly_rent].shape[0]

11360

I also noticed ‘TotalPay’ is a combination of ‘BasePay’ and ‘OvertimePay’. I wondered how many city employee’s *needed* to work overtime to afford to live.

Since the ‘BasePay’ column didn’t properly import as a number, we must do some conversion first.

After converting, I took a rough guess if you spend 70% of your income on rent, you can’t afford to live there. 

In [27]:
pd.to_numeric(latest_salaries['BasePay'], errors='coerce').describe()

count     38119.000000
mean      66564.421924
std       44053.783972
min           0.000000
25%       30138.055000
50%       65055.000000
75%       94735.050000
max      318835.490000
Name: BasePay, dtype: float64

In [31]:
# Get the column of BasePay and make sure the elements in the columns are numerical
base_pay_series = pd.to_numeric(latest_salaries['BasePay'], errors='coerce')

# Compute the number of employees that have to spend more than 70% their income on rent
base_pay_series[base_pay_series * 0.7 < average_yearly_rent].shape


(20074,)

I wanted to see how many city employees were working more than $1K overtime annually.

In [32]:
overtime_series = pd.to_numeric(latest_salaries['OvertimePay'], errors='coerce')

overtime_series[overtime_series > 1000].shape

(15361,)

I want to explore which types of employees make more or less money

In [59]:
# Using scipy's mode function which ignores NaN values
from scipy.stats import mode

# Average income of employees
average_totalpay = latest_salaries['TotalPay'].mean()

# Create DataFrame for employees whose TotalPay above/below than Average TotalPay
above_average = latest_salaries[latest_salaries['TotalPay'] > average_totalpay]
below_average = latest_salaries[latest_salaries['TotalPay'] < average_totalpay]

# Print the number of employees whose TotalPay above/below than Average TotalPay
print('Number of employees whose income aboves the average', above_average.shape[0])
print('Number of employees whose income belows the average', below_average.shape[0])

# Get the number of the most frequent job in two cases
print('Job Title above', mode(above_average['JobTitle']))
print('Job Title below', mode(below_average['JobTitle']))

# Get the frequency of job titles in two cases
print(above_average['JobTitle'].value_counts())
print(below_average['JobTitle'].value_counts())

('Number of employees whose income aboves the average', 17978)
('Number of employees whose income belows the average', 20145)
('Job Title above', ModeResult(mode=array(['Registered Nurse'], dtype=object), count=array([1065])))
('Job Title below', ModeResult(mode=array(['Transit Operator'], dtype=object), count=array([1485])))
Registered Nurse                  1065
Transit Operator                   994
Firefighter                        748
Police Officer 3                   714
Deputy Sheriff                     586
Police Officer                     411
Sergeant 3                         363
Attorney (Civil/Criminal)          337
Police Officer 2                   323
EMT/Paramedic/Firefighter          259
Transit Supervisor                 199
Special Nurse                      193
Electrical Transit System Mech     191
Lieutenant, Fire Suppression       190
Protective Services Worker         171
Assoc Engineer                     170
Stationary Engineer                168
Senior Ad

I want to check whether employees are getting normal pay raises and promotions through the year 2011 to 2014. I assumed that in DataFrame salaries, the rows with the same value of "EmployeeName" and "JobTitle" indicate the same person. I have chosen a specific person and check his promotions from 2011 to 2014. 

In [96]:
# Having a look at the employee name
print(salaries['EmployeeName'].value_counts().head())

# Choose the guy named 'Kevin Lee'
random_employee = salaries[salaries['EmployeeName'] == 'Kevin Lee']

# Have a look at the JobTiles go with 'Kevin Lee'
random_employee[['EmployeeName', 'JobTitle']]

Kevin Lee       13
William Wong    11
Richard Lee     11
Steven Lee      11
Michael Wong     9
Name: EmployeeName, dtype: int64


,EmployeeName,JobTitle
Id,,
39716,Kevin Lee,Police Officer 3
40571,Kevin Lee,Police Officer 3
42511,Kevin Lee,Electrical Transit System Mech
49271,Kevin Lee,Deputy Court Clerk III
49440,Kevin Lee,Deputy Court Clerk III
50124,Kevin Lee,Personnel Analyst
52234,Kevin Lee,Senior Parking Control Officer
53932,Kevin Lee,Transit Operator
55120,Kevin Lee,IS Administrator 2


In [129]:
# Choose the guy 'Kevin Lee' who is a 'Transit Operator'
kevin = random_employee[random_employee['JobTitle'] == 'Transit Operator']

# Compute the percentage of Kevin's promotion
promotion_percentage = kevin[kevin['Year']==2014]['TotalPay'].values/kevin[kevin['Year']==2012]['TotalPay'].values
print('%f%%' % (promotion_percentage*100))

118.868647%


We could also do more research to determine whether the average family household in San Francisco has two incomes and what those are, as well as how many bedrooms the average family in San Francisco has. We could figure out more average pay wages in San Francisco (how much do teachers make? Cab drivers? What about manual labor? Restaurant staff?). We could also map the dataset against the Priceonomics data to show what neighborhoods the average city employee can afford and how much longer their commute is due to the rent increases. We could calculate more about the cost of living using many sites tracking average cost of living, and build a salary converter to show how much you would need to make in San Francisco to support your quality of life. 

References: http://dataconomy.com/2016/10/big-data-python/